In [1]:
import cv2
import numpy as np

In [2]:
#reading the image as gray scale. Image is present in the same folder as this file
img = cv2.imread('edge.png', 0)
[rows,cols] = img.shape
xgrad = [[0] * (cols+2) for _ in range(rows+2)]
ygrad = [[0] * (cols+2) for _ in range(rows+2)]
xgrad = np.asarray(xgrad)
ygrad = np.asarray(ygrad)

In [3]:
#padding values to calculate x gradient
listofzeros = [0] * (cols+2)
image_pad_x = list()
image_pad_x.append(listofzeros)
for i in img:
    i = list(i)
    i.insert(0,0)
    i.insert(len(i)+1,0)
    image_pad_x.append(i)
image_pad_x.append(listofzeros)
image_pad_x = np.asarray(image_pad_x)

In [4]:
#calculating gradient image in X-direction
for i in range(1,len(image_pad_x)-2):
    for j in range(1,len(image_pad_x[i])-2):
        xgrad[i][j] = image_pad_x[i+1][j] - image_pad_x[i-1][j]

cv2.imwrite('image_edge_xgrad.png', np.asarray(xgrad))

#calculating gradient image in Y-direction
for i in range(1,len(image_pad_x)-2):
    for j in range(1,len(image_pad_x[1])-2):
        ygrad[i][j] = image_pad_x[i][j+1] - image_pad_x[i][j-1]
cv2.imwrite('image_edge_ygrad.png', np.asarray(ygrad))

True

In [13]:
#Defining Sobel Operator
x_sob = [[1, 0, -1], 
         [2, 0, -2], 
         [1, 0, -1]]
y_sob = [[1, 2, 1], 
         [0, 0, 0], 
         [-1, -2, -1]]
#Flipping the kernel for convolution
for i in x_sob:
    temp = i[0]
    i[0] = i[2]
    i[2] = temp

temp = y_sob[0]
y_sob[0] = y_sob[2]
y_sob[2] = temp

x_sob = np.asarray(x_sob)
y_sob = np.asarray(y_sob)
#creating numpy arrays to store the result of convolution
sobelxImage = [ [0.0] * (cols) for _ in range(rows)]
sobelyImage = [ [0.0] * (cols) for _ in range(rows)]
sobelxImage = np.asarray(sobelxImage)
sobelyImage = np.asarray(sobelyImage)

In [14]:
#Performing the convolution
for i in range(1,rows-1):
    for j in range(1,cols-1):       
        #Calculate gx and gy using Sobel (horizontal and vertical gradients)
        gx = (x_sob[0,0] * xgrad[i-1,j-1]) + (x_sob[0,1] * xgrad[i-1,j]) + \
             (x_sob[0,2] * xgrad[i-1,j+1]) + (x_sob[1,0] * xgrad[i,j-1]) + \
             (x_sob[1,1] * xgrad[i,j]) + (x_sob[1,2] * xgrad[i,j+1]) + \
             (x_sob[2,0] * xgrad[i+1,j-1]) + (x_sob[2,1] * xgrad[i+1,j]) + \
             (x_sob[2,2] * xgrad[i+1,j+1])

        gy = (y_sob[0,0] * ygrad[i-1,j-1]) + (y_sob[0,1] * ygrad[i-1,j]) + \
             (y_sob[0,2] * ygrad[i-1,j+1]) + (y_sob[1,0] * ygrad[i,j-1]) + \
             (y_sob[1,1] * ygrad[i,j]) + (y_sob[1,2] * ygrad[i,j+1]) + \
             (y_sob[2,0] * ygrad[i+1,j-1]) + (y_sob[2,1] * ygrad[i+1,j]) + \
             (y_sob[2,2] * ygrad[i+1,j+1])
        sobelxImage[i-1,j-1] = gx
        sobelyImage[i-1,j-1] = gy
print("Finished convolution")

Finished convolution


In [15]:
cv2.imwrite('image_edge_sobelxImage.png', sobelxImage)
cv2.imwrite('image_edge_sobelyImage.png', sobelyImage)

True

In [16]:
#Normalising the convoluted image to remove the negative values and to restrict the range of values from 0 to 255
maximum_x = sobelxImage[0,0]
for i in range(len(sobelxImage)):
    for j in range(len(sobelxImage[i])):
        if(maximum_x < abs(sobelxImage[i,j])):
            maximum_x = abs(sobelxImage[i,j])

maximum_y = sobelyImage[0,0]
for i in range(len(sobelyImage)):
    for j in range(len(sobelyImage[i])):
        if(maximum_y < abs(sobelyImage[i,j])):
            maximum_y = abs(sobelyImage[i,j])

In [17]:
edge_x = [ [0.0] * (cols) for _ in range(rows)]
for i in range(len(edge_x)):
    for j in range(len(edge_x[i])):        
        edge_x[i][j] = (float)(abs(sobelxImage[i,j])*255/maximum_x)
edge_x = np.asarray(edge_x)

In [18]:
edge_y = [ [0.0] * (cols) for _ in range(rows)]
for i in range(len(edge_y)):
    for j in range(len(edge_y[i])):        
        edge_y[i][j] = abs(sobelyImage[i,j])*255/maximum_y
edge_y = np.asarray(edge_y)

In [19]:
cv2.imwrite('image_edge_x_norm.png', edge_x)

True

In [20]:
cv2.imwrite('image_edge_y_norm.png', edge_y)

True